In [80]:
import duckdb

In [81]:
con = duckdb.connect('../localdatabase.db')
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

In [82]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'marketing-and-ecommerce',
    'endpoint': 'localhost:9000'
}

# --- 1. Konfigurasi MinIO (S3 API) ---
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;")
con.sql("SET s3_url_style = 'path';")

In [83]:
dbPath = '../datalakehouse.db'

In [84]:
con.sql(f"ATTACH 'ducklake:{dbPath}' AS datalakehouse;")
con.sql("USE datalakehouse;")

In [85]:
con.sql("SELECT * FROM ducklake_snapshots('datalakehouse')")

┌─────────────┬───────────────────────────────┬────────────────┬──────────────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                                 changes                                  │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                         map(varchar, varchar[])                          │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼──────────────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-14 21:27:19.613549+07 │              0 │ {schemas_created=[main]}                                                 │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-12-14 21:27:19.753413+07 │              1 │ 

In [86]:
con.sql("SELECT * FROM raw.campaigns")

┌─────────────┬─────────────┬──────────────┬────────────┬────────────┬────────────────┬─────────────────┐
│ campaign_id │   channel   │  objective   │ start_date │  end_date  │ target_segment │ expected_uplift │
│    int32    │   varchar   │   varchar    │    date    │    date    │    varchar     │      float      │
├─────────────┼─────────────┼──────────────┼────────────┼────────────┼────────────────┼─────────────────┤
│           1 │ Paid Search │ Cross-sell   │ 2021-10-25 │ 2021-11-26 │ Deal Seekers   │           0.022 │
│           2 │ Email       │ Retention    │ 2021-10-24 │ 2021-12-24 │ Deal Seekers   │           0.116 │
│           3 │ Email       │ Reactivation │ 2023-10-08 │ 2023-11-30 │ Churn Risk     │             0.1 │
│           4 │ Display     │ Reactivation │ 2022-07-25 │ 2022-10-07 │ Deal Seekers   │           0.111 │
│           5 │ Social      │ Acquisition  │ 2022-07-09 │ 2022-09-29 │ New Customers  │           0.144 │
│           6 │ Affiliate   │ Reactivation │ 2

In [87]:
con.sql("SELECT * FROM raw.customers")

┌─────────────┬─────────────┬─────────┬───────┬─────────┬──────────────┬─────────────────────┐
│ customer_id │ signup_date │ country │  age  │ gender  │ loyalty_tier │ acquisition_channel │
│    int32    │    date     │ varchar │ int32 │ varchar │   varchar    │       varchar       │
├─────────────┼─────────────┼─────────┼───────┼─────────┼──────────────┼─────────────────────┤
│           1 │ 2021-04-08  │ BR      │    48 │ Male    │ Bronze       │ Referral            │
│           2 │ 2023-04-28  │ IN      │    36 │ Female  │ Silver       │ Organic             │
│           3 │ 2022-12-18  │ UK      │    35 │ Female  │ Silver       │ Organic             │
│           4 │ 2022-04-26  │ US      │    45 │ Male    │ Silver       │ Paid Search         │
│           5 │ 2022-04-20  │ IN      │    53 │ Male    │ Silver       │ Organic             │
│           6 │ 2023-07-30  │ IN      │    31 │ Female  │ Bronze       │ Paid Search         │
│           7 │ 2021-04-05  │ IN      │    35 │ Fe

In [88]:
con.sql("SELECT * FROM raw.events")

┌──────────┬─────────────────────┬─────────────┬────────────┬─────────────┬────────────┬─────────────┬────────────────┬─────────────┬───────────────┬──────────────────────┬──────────────────┐
│ event_id │      timestamp      │ customer_id │ session_id │ event_type  │ product_id │ device_type │ traffic_source │ campaign_id │ page_category │ session_duration_sec │ experiment_group │
│  int32   │      timestamp      │    int32    │   int32    │   varchar   │   int32    │   varchar   │    varchar     │    int32    │    varchar    │        float         │     varchar      │
├──────────┼─────────────────────┼─────────────┼────────────┼─────────────┼────────────┼─────────────┼────────────────┼─────────────┼───────────────┼──────────────────────┼──────────────────┤
│    67001 │ 2021-09-18 04:46:27 │       56506 │     604161 │ purchase    │        873 │ mobile      │ Organic        │           0 │ PLP           │                 40.5 │ Variant_B        │
│    67002 │ 2021-05-04 20:15:26 │      

In [89]:
con.sql("SELECT * FROM raw.products")

┌────────────┬─────────────┬───────────┬────────────┬─────────────┬────────────┐
│ product_id │  category   │   brand   │ base_price │ launch_date │ is_premium │
│   int32    │   varchar   │  varchar  │   float    │    date     │   int32    │
├────────────┼─────────────┼───────────┼────────────┼─────────────┼────────────┤
│          1 │ Grocery     │ Brand_58  │      14.19 │ 2021-08-02  │          0 │
│          2 │ Fashion     │ Brand_1   │       25.8 │ 2021-09-14  │          0 │
│          3 │ Electronics │ Brand_70  │     165.46 │ 2021-01-18  │          1 │
│          4 │ Fashion     │ Brand_56  │      75.45 │ 2023-03-03  │          1 │
│          5 │ Sports      │ Brand_1   │       72.5 │ 2022-04-19  │          1 │
│          6 │ Fashion     │ Brand_39  │      57.08 │ 2023-09-16  │          0 │
│          7 │ Beauty      │ Brand_42  │      27.23 │ 2023-05-17  │          0 │
│          8 │ Home        │ Brand_41  │     121.82 │ 2021-02-16  │          1 │
│          9 │ Home        │

In [90]:
con.sql("SELECT * FROM raw.transactions")

┌────────────────┬─────────────────────┬─────────────┬────────────┬──────────┬──────────────────┬───────────────┬─────────────┬─────────────┐
│ transaction_id │      timestamp      │ customer_id │ product_id │ quantity │ discount_applied │ gross_revenue │ campaign_id │ refund_flag │
│     int32      │      timestamp      │    int32    │   int32    │  int32   │      float       │     float     │    int32    │    int32    │
├────────────────┼─────────────────────┼─────────────┼────────────┼──────────┼──────────────────┼───────────────┼─────────────┼─────────────┤
│              1 │ 2021-12-27 08:25:15 │       59540 │       1630 │        3 │              0.0 │         43.74 │           0 │           0 │
│              2 │ 2023-06-06 21:14:26 │       54871 │       1901 │        3 │              0.0 │        174.78 │          21 │           0 │
│              3 │ 2023-08-31 05:29:54 │       51818 │       1884 │        1 │              0.0 │         40.61 │          37 │           0 │
│     

In [91]:
con.sql("SELECT * FROM datalakehouse.model.campaign_performance")

┌──────────────┬─────────────┬────────────────────┬────────────────────┬───────────────────────┬──────────────────────┐
│  objective   │   channel   │ total_net_revenue  │ total_transactions │ avg_discount_rate_pct │     refund_rate      │
│   varchar    │   varchar   │       double       │       int64        │        double         │        double        │
├──────────────┼─────────────┼────────────────────┼────────────────────┼───────────────────────┼──────────────────────┤
│ Cross-sell   │ Paid Search │  827187.7883341312 │              10461 │      4.06605495972725 │  0.02934709874772966 │
│ Reactivation │ Affiliate   │  534271.9231247902 │               6911 │     4.283750635931451 │ 0.035161336998987124 │
│ Retention    │ Affiliate   │  489687.9131574631 │               6091 │     4.133968242277283 │ 0.025447381382367427 │
│ Reactivation │ Email       │  471952.0583062172 │               6031 │      4.12783958660224 │ 0.026197977118222516 │
│ Reactivation │ Display     │  397051.7

In [92]:
con.close()